#### Paquetes Necesarios

In [2]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import pyglet
import time
from pydub import AudioSegment

#### Inicialización del Detector y Utilidades

In [3]:
detector = HandDetector(staticMode=False, maxHands=2, modelComplexity=1, detectionCon=0.5, minTrackCon=0.5)

keys = ["A", "B", "C", "D", "E", "F", "G"]

class Button():
    def __init__(self, pos, text, size, color, border=2):
        self.pos = pos
        self.size = size
        self.text = text
        self.color = color
        self.border = border
        self.isPressed = False
        self.lastPressTime = 0

buttonList = []
for j, key in enumerate(keys):
    buttonList.append(Button([(70 * j) + 400, 50], key, [60, 140], (255, 255, 255)))

current_sequence = [] 
saveButton = Button([100, 100], "Save", [100, 50], (255, 255, 255))
save_counter = 1 

logo = cv2.imread('./images/piano.png', cv2.IMREAD_UNCHANGED)
logo = cv2.resize(logo, (100, 100)) 

#### Método para Tocar Teclas

In [4]:
def playkeys(button):
    current_time = time.time()
    
    if not button.isPressed and (current_time - button.lastPressTime) > 1.0:
        audio_filename = f"./notes/{button.text}.wav"  # Nombre del archivo de sonido
        pyglet.media.load(audio_filename).play()
        button.isPressed = True
        button.lastPressTime = current_time
        current_sequence.append(button.text)

#### Método para Guardar la Secuencia

In [5]:
def save_sequence(sequence, filename="output_sequence"):
    global save_counter
    
    if len(sequence) > 0:
        melody_segments = []

        for note in sequence:
            audio_filename = f"./notes/{note}.wav"  
            sound = AudioSegment.from_file(audio_filename, format="wav")
            melody_segments.append(sound)

        melody = sum(melody_segments)
        
        melody_filename = f"{filename}_{save_counter}.wav"
        melody.export(melody_filename, format="wav")

        save_counter += 1  

#### Método para Resetear la Secuencia

In [6]:
def reset_sequence():
    global current_sequence
    current_sequence = []

#### Método para Dibujar el Instrumento

In [7]:
def drawAll(img, buttonList):
    cv2.rectangle(img, (350, 25 ), (910, 200), (0, 0, 0), cv2.FILLED)
    for button in buttonList:
        x, y = button.pos
        w, h = button.size
        colorr = button.color
        border = button.border

        cv2.rectangle(img, button.pos, (x + w, y + h), colorr, cv2.FILLED)
        cv2.rectangle(img, button.pos, (x + w, y + h), (0, 0, 0), border)
        cv2.putText(img, button.text, (x + 10, y + h - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (214, 0, 220), 2)
    
    x, y = saveButton.pos
    w, h = saveButton.size
    colorr = saveButton.color
    border = saveButton.border
    cv2.rectangle(img, saveButton.pos, (x + w, y + h), colorr, cv2.FILLED)
    cv2.rectangle(img, saveButton.pos, (x + w, y + h), (0, 0, 0), border)
    cv2.putText(img, saveButton.text, (x + 10, y + h - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (214, 0, 220), 2)
    
    return img

#### Bucle Principal

In [9]:
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

while True:
    success, img = cap.read()

    img = cv2.flip(img, 1)

    hands, img = detector.findHands(img, draw=True, flipType=False)

    for button in buttonList:
        button.isPressed = False
        button.color = (255, 255, 255)  

    saveButton.isPressed = False
    saveButton.color = (255, 255, 255)

    for hand in hands:
        lmList = hand["lmList"]
        for button in buttonList:
            x, y = button.pos
            w, h = button.size

            if x < lmList[8][0] < x + w and y < lmList[8][1] < y + h:
                button.color = (0, 255, 0)
                playkeys(button)
                button.isPressed = True

            x, y = saveButton.pos
            w, h = saveButton.size
            if (x < lmList[8][0] < x + w and y < lmList[8][1] < y + h) or (x < lmList[12][0] < x + w and y < lmList[12][1] < y + h):
                if current_sequence:
                    saveButton.isPressed = True
                    saveButton.color = (0, 255, 0)
                    save_sequence(current_sequence)
                    reset_sequence()            

    img = drawAll(img, buttonList)

    rows, cols, channels = logo.shape
    roi = img[0:rows, img.shape[1] - cols:img.shape[1]]
    img_bg = cv2.bitwise_and(roi, roi, mask=cv2.bitwise_not(logo[:, :, 3]))
    img_fg = cv2.bitwise_and(logo[:, :, 0:3], logo[:, :, 0:3], mask=logo[:, :, 3])
    dst = cv2.addWeighted(img_bg, 1, img_fg, 1, 0)
    img[0:rows, img.shape[1] - cols:img.shape[1]] = dst

    cv2.imshow("Virtual Piano", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()